In [1]:
import sys
import tqdm
import time
import sklearn
import numpy as np
import pandas as pd
import scipy
import copy
import random
import math
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from load_dataset import load, generate_random_dataset
from classifier import NeuralNetwork, LogisticRegression, SVM
from utils import *
from metrics import *  # include fairness and corresponding derivatives
from scipy import stats
from scipy.stats import rankdata
from sklearn import metrics, preprocessing
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
from operator import itemgetter
from torch.autograd import grad
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

In [2]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore') 

**Load Dataset**

In [3]:
dataset = 'german'
X_train, X_test, y_train, y_test = load(dataset)

**Parametric Model**

In [5]:
X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Loss function** (Log loss for logistic regression)

In [7]:
# clf = NeuralNetwork(input_size=X_train.shape[-1])
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])
num_params = len(convert_grad_to_ndarray(list(clf.parameters())))
if isinstance(clf, LogisticRegression) or isinstance(clf, NeuralNetwork):
    loss_func = logistic_loss_torch
elif isinstance(clf, SVM):
    loss_func = svm_loss_torch

In [8]:
def logistic_loss_new():
    y_pred = model(x)
    return binary_cross_entropy(y_pred, torch.Tensor([y_true]))

**Influence of points computed using ground truth**

In [9]:
def ground_truth_influence(X_train, y_train, X_test, X_test_orig, y_test):
    clf.fit(X_train, y_train, verbose=True)
    y_pred = clf.predict_proba(X_test)
    spd_0 = computeFairness(y_pred, X_test_orig, y_test, 0)

    delta_spd = []
    for i in range(len(X_train)):
        X_removed = np.delete(X_train, i, 0)
        y_removed = y_train.drop(index=i, inplace=False)
        clf.fit(X_removed, y_removed)
        y_pred = clf.predict_proba(X_test)
        delta_spd_i = computeFairness(y_pred, X_test_orig, y_test, 0) - spd_0
        delta_spd.append(delta_spd_i)

    return delta_spd

**Compute Accuracy** 

In [10]:
def computeAccuracy(y_true, y_pred):
    return np.sum((y_pred>0.5) == y_true)/len(y_pred)

**First-order derivative of loss function at z with respect to model parameters**

In [11]:
def del_L_del_theta_i(model, x, y_true, retain_graph=False):
    loss = loss_func(model, x, y_true)
    w = [ p for p in model.parameters() if p.requires_grad ]
    return grad(loss, w, create_graph=True, retain_graph=retain_graph)

**First-order derivative of P(y | x) with respect to model parameters**

In [12]:
def del_f_del_theta_i(model, x, retain_graph=False):
    w = [ p for p in model.parameters() if p.requires_grad ]
    return grad(model(torch.FloatTensor(x)), w, retain_graph=retain_graph)

**Stochastic estimation of Hessian vector product (involving del fairness): $H_{\theta}^{-1}v = H_{\theta}^{-1}\nabla_{\theta}f(z, \theta) = v + [I - \nabla_{\theta}^2L(z_{s_j}, \theta^*)]H_{\theta}^{-1}v$**

In [13]:
def hvp(y, w, v):
    ''' Multiply the Hessians of y and w by v.'''
    # First backprop
    first_grads = grad(y, w, retain_graph=True, create_graph=True)

    # Elementwise products
    elemwise_products = 0
    for grad_elem, v_elem in zip(convert_grad_to_tensor(first_grads), v):
        elemwise_products += torch.sum(grad_elem * v_elem)

    # Second backprop
    return_grads = grad(elemwise_products, w, create_graph=True)

    return return_grads

In [14]:
def hessian_one_point(model, x, y):
    x, y = torch.FloatTensor(x), torch.FloatTensor([y])
    loss = loss_func(model, x, y)
    params = [ p for p in model.parameters() if p.requires_grad ]
    first_grads = convert_grad_to_tensor(grad(loss, params, retain_graph=True, create_graph=True))
    hv = np.zeros((len(first_grads), len(first_grads)))
    for i in range(len(first_grads)):
        hv[i, :] = convert_grad_to_ndarray(grad(first_grads[i], params, create_graph=True)).ravel()
    return hv

In [15]:
# Compute multiplication of inverse hessian matrix and vector v
def s_test(model, xs, ys, v, hinv=None, damp=0.01, scale=25.0, r=-1, batch_size=-1, recursive=False, verbose=False):
    ''' Arguments:
        xs: list of data points
        ys: list of true labels corresponding to data points in xs
        damp: dampening factor
        scale: scaling factor
        r: number of iterations aka recursion depth
            should be enough so that the value stabilises.
        batch_size: number of instances in each batch in recursive approximation
        recursive: determine whether to recursively approximate hinv_v'''
    xs, ys = torch.FloatTensor(xs.copy()), torch.FloatTensor(ys.copy())
    n = len(xs)
    if recursive:
        hinv_v = copy.deepcopy(v)
        if verbose:
            print('Computing s_test...')
            tbar = tqdm.tqdm(total=r)
        if (batch_size == -1):  # default
            batch_size = 10
        if (r == -1):
            r = n // batch_size + 1
        sample = np.random.choice(range(n), r*batch_size, replace=True)
        for i in range(r):
            sample_idx = sample[i*batch_size:(i+1)*batch_size]
            x, y = xs[sample_idx], ys[sample_idx]
            loss = loss_func(model, x, y)
            params = [ p for p in model.parameters() if p.requires_grad ]
            hv = convert_grad_to_ndarray(hvp(loss, params, torch.FloatTensor(hinv_v)))
            # Recursively caclulate h_estimate
            hinv_v = v + (1 - damp) * hinv_v - hv / scale
            if verbose:
                tbar.update(1)
    else:
        if hinv is None:
            hinv = np.linalg.pinv(np.sum(hessian_all_points, axis=0))
        scale = 1.0
        hinv_v = np.matmul(hinv, v)

    return hinv_v / scale

Update Start here

In [16]:
# Learning Rate (can try with different step sizes and choose the best)
n = 50

In [17]:
# The pattern to be updated
idx = (X_train_orig.age==1)&(X_train_orig.gender==0)
delta_hist = []
final_out_hist = []
S = torch.Tensor(X_train[idx])
S.requires_grad = True
delta = torch.zeros(1, X_train.shape[-1])
delta.requires_grad = True
S_new = S + delta

In [29]:
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = NeuralNetwork(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])

clf.fit(X_train, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
print("Initial statistical parity: ", spd_0)

tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
print("Initial TPR parity: ", tpr_parity_0)

predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.09527580118738121
Initial TPR parity:  -0.07785149359511678
Initial predictive parity:  -0.10136869102808022
Initial loss:  0.5078892147492744
Initial accuracy:  0.755


In [19]:
metric = 0
if metric == 0:
    v1 = del_spd_del_theta(clf, X_test_orig, X_test, dataset)
elif metric == 1:
    v1 = del_tpr_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)
elif metric == 2:
    v1 = del_predictive_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)

In [20]:
part_1 = torch.FloatTensor(v1).repeat(len(S_new), 1).reshape(len(S_new), 1, -1)

In [21]:
part_2 = []
for i in range(len(S_new)):
    inner_lst = []
    del_L_del_theta_i_t = convert_grad_to_tensor(del_L_del_theta_i(clf, S_new[i], y_train[i], retain_graph=True))
    for j in range(len(del_L_del_theta_i_t)):
        inner_grad = convert_grad_to_ndarray(grad(del_L_del_theta_i_t[j], delta, retain_graph=True))
        inner_lst.append(inner_grad)
    part_2.append(np.array(inner_lst))
    
part_2 = np.array(part_2)
part_2 = torch.FloatTensor(part_2)
part_2 = part_2.mean(dim=0).unsqueeze(0).repeat(len(S_new), 1, 1)
# part_2 = part_2.mean(dim=0).unsqueeze(0)

In [22]:
final = torch.bmm(part_1, part_2).reshape((len(S_new), -1))
delta = delta - n*final
S_new = S_new + delta

In [23]:
X_train_new = X_train.copy()
X_train_new[idx] = X_train_new[idx] + delta.detach().numpy()

In [24]:
from scipy.optimize import Bounds, minimize

In [25]:
mins = []
maxs = []
numCols = len(X_train[0])
new_S = []
for i in range(numCols):
    mins.insert(i, min(X_train[:, i]))
    maxs.insert(i, max(X_train[:, i]))

bounds = Bounds(mins, maxs)
tbar = tqdm.tqdm(total=len(S_new))
for i in X_train_orig[idx].index:
    X_train_pert_pt = X_train_new[i]
    f = lambda x: np.linalg.norm(x - X_train_pert_pt)

    x0 = np.random.rand(numCols)
    res = minimize(f, x0, method='trust-constr', options={'verbose': 0}, bounds=bounds)
    X_train_new[i] = res.x
    tbar.update(1)

100%|██████████| 40/40 [00:10<00:00,  3.74it/s]

In [26]:
expl_ = ["age=0", "install_rate=4", "residence=2"]
def get_subset(explanation):
    subset = X_train_orig.copy()
    for predicate in explanation:
        attr = predicate.split("=")[0].strip(' ')
        val = int(predicate.split("=")[1].strip(' '))
        subset = subset[subset[attr] == val]
    return subset.index
X_train_orig.loc[get_subset(expl_)]

,status,duration,credit_hist,credit_amt,savings,employment,install_rate,debtors,residence,property,...,purpose_A42,purpose_A43,purpose_A44,purpose_A45,purpose_A46,purpose_A48,purpose_A49,housing_A151,housing_A152,housing_A153
16,1,1,2,0,0,1,4,0,2,3,...,0,0,0,0,0,0,0,0,1,0
23,3,1,0,0,0,2,4,0,2,0,...,1,0,0,0,0,0,0,0,1,0
28,0,1,2,2,0,0,4,0,2,0,...,0,0,0,0,0,0,0,0,0,1
33,1,1,0,0,0,2,4,0,2,1,...,0,1,0,0,0,0,0,0,1,0
34,1,2,1,1,1,4,4,0,2,2,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,3,0,2,0,0,3,4,0,2,1,...,0,1,0,0,0,0,0,1,0,0
784,3,1,2,0,0,2,4,0,2,2,...,0,1,0,0,0,0,0,0,1,0
791,3,1,2,0,0,1,4,0,2,3,...,0,0,0,0,0,0,0,0,1,0
798,0,1,2,0,0,0,4,0,2,1,...,0,1,0,0,0,0,0,0,1,0


In [27]:
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = NeuralNetwork(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])

clf.fit(X_train_new, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
print("Initial statistical parity: ", spd_0)

tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
print("Initial TPR parity: ", tpr_parity_0)

predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.018946960102766752
Initial TPR parity:  -0.008778598193728615
Initial predictive parity:  -0.11441340371232289
Initial loss:  0.5159324960607171
Initial accuracy:  0.775
